In [ ]:
!unzip "/content/drive/My Drive/Colab Notebooks/intel-data.zip"

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.initializers import glorot_uniform
import os

class EndTraining(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epochs, logs=None):
    if logs.get('accuracy')>=0.98 and logs.get('val_accuracy')>=0.98:
      self.model.stop_training=True


In [ ]:
class ResNet():

  def __init__(self, model=None):
    self.model = model


  def loadData(self, train_dir, test_dir, inputShape, nClasses, batchSize=64):
    ''' 
    Arguments:
    ----------
    train_dir => Path for the directory containing training images.
    test_dir => Path for the directory containing testing images.
    inputShape => Shape of the image.
    nClasses => No. of classes, the dataset is divided into.
    batchSize => No. of images to be put in one batch

    Description:
    ------------
    The function generates train and test generators from the directory paths given as inputs.
    '''
    self.inputShape = inputShape
    self.nClasses = nClasses

    train_image_generator = ImageDataGenerator(rescale=1/255)
    test_image_generator = ImageDataGenerator(rescale=1/255) 

    self.train_data_gen = train_image_generator.flow_from_directory(batch_size=batchSize,
                                                              directory=train_dir,
                                                              shuffle=True,
                                                              target_size=(inputShape[0], inputShape[1]),
                                                              class_mode='categorical')

    self.test_data_gen = test_image_generator.flow_from_directory(batch_size=batchSize,
                                                              directory=test_dir,
                                                              shuffle=True,
                                                              target_size=(inputShape[0], inputShape[1]),
                                                              class_mode='categorical')


  def train(self, epochs):
    ''' 
    Arguments:
    ----------
    epochs => No. of iterations for which training should continue.

    Output:
    -------
    hist => History of the trained model.
    '''
    
    # If self.model==None, then train a new model, else continue training the model that is passed as an argument to the initializer
    if self.model == None:
      self.resNet50() 
    self.model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    callback = EndTraining()
    hist = self.model.fit(self.train_data_gen, epochs = epochs, validation_data=self.test_data_gen, callbacks=[callback])
    return hist


  def predict(self, imagePath):
    ''' 
    Arguments:
    ----------
    imagePath => Path of the input image that needs to be classified.

    Output:
    -------
    index => Index of the class to which the image is classified into.
    '''

    img = image.load_img(img_path, target_size=self.inputShape)
    x = image.img_to_array(img)

    # Add a dimension that indicates batch number
    x = np.expand_dims(x, axis=0)

    # Normalize the image
    x = x/255.0

    index = self.model.predict(x)
    return index


  def resNet50(self):
    '''  
    Output:
    -------
    model => A model instance in Keras

    Description:
    ------------
    Implementation of the popular ResNet50 the following architecture:
      (CONV2D -> BATCHNORM -> RELU -> MAXPOOL) -> (CONVBLOCK -> IDBLOCK*2) -> (CONVBLOCK -> IDBLOCK*3)
      -> (CONVBLOCK -> IDBLOCK*5) -> (CONVBLOCK -> IDBLOCK*2) -> AVGPOOL -> TOPLAYER
    '''

    # Define the input as a tensor with shape inputShape
    X_input = Input(self.inputShape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)

    # Stage 1: (CONV2D -> BATCHNORM -> RELU -> MAXPOOL)
    X = Conv2D(filters=64, kernel_size=(7, 7), strides = (2, 2), kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2: (CONVBLOCK -> IDBLOCK*2)
    X = self.convBlock(X, filterSize=3, numFilters=[64, 64, 256], stride = 1)
    X = self.identityBlock(X, filterSize=3, numFilters=[64, 64, 256])
    X = self.identityBlock(X, filterSize=3, numFilters=[64, 64, 256])

    # Stage 3: (CONVBLOCK -> IDBLOCK*3)
    X = self.convBlock(X, filterSize=3, numFilters=[128, 128, 512], stride = 2)
    X = self.identityBlock(X, filterSize=3, numFilters=[128, 128, 512])
    X = self.identityBlock(X, filterSize=3, numFilters=[128, 128, 512])
    X = self.identityBlock(X, filterSize=3, numFilters=[128, 128, 512])

    # Stage 4: (CONVBLOCK -> IDBLOCK*5)
    X = self.convBlock(X, filterSize=3, numFilters=[256, 256, 1024], stride = 2)
    X = self.identityBlock(X, filterSize=3, numFilters=[256, 256, 1024])
    X = self.identityBlock(X, filterSize=3, numFilters=[256, 256, 1024])
    X = self.identityBlock(X, filterSize=3, numFilters=[256, 256, 1024])
    X = self.identityBlock(X, filterSize=3, numFilters=[256, 256, 1024])
    X = self.identityBlock(X, filterSize=3, numFilters=[256, 256, 1024])

    # Stage 5: (CONVBLOCK -> IDBLOCK*2)
    X = self.convBlock(X, filterSize=3, numFilters=[512, 512, 2048], stride = 2)
    X = self.identityBlock(X, filterSize=3, numFilters=[512, 512, 2048])
    X = self.identityBlock(X, filterSize=3, numFilters=[512, 512, 2048])

    # AvgPool
    X = AveragePooling2D((2, 2))(X)

    # Dense Layer
    X = Flatten()(X)
    X = Dense(self.nClasses, activation='softmax', kernel_initializer = glorot_uniform(seed=0))(X)

    # With the "Functional API", where you start from Input, you chain layer calls to specify the model's forward pass, 
    # and finally you create your model from inputs and outputs
    self.model = Model(inputs = X_input, outputs = X)


  def identityBlock(self, X, filterSize, numFilters):
    ''' 
    Arguments:
    ----------
    X => Input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    filterSize => Size of the filter for the middle Conv Layer
    numFilters => Python list of integers, defining the number of filters in the CONV layers

    Output:
    -------
    X => output of the identity block, tensor of shape (n_H, n_W, n_C)

    Description:
    ------------
    Following is the identity block:
      X - conv-batchNorm-relu1 - conv-batchNorm-relu2 - conv-batchNorm3 - relu1
      X_dash - relu1
    '''

    # Retrieve number of filters for each Conv Layer
    nF1, nF2, nF3 = numFilters

    # Save the input value so that it can be fed directly to the last relu layer of the block
    X_dash = X

    # First component - conv-batchNorm-relu1
    X = Conv2D(filters = nF1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)

    # Second component - conv-batchNorm-relu2
    X = Conv2D(filters = nF2, kernel_size = (filterSize, filterSize), strides = (1,1), padding = 'same', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)

    # Third component - conv-batchNorm3
    X = Conv2D(filters = nF3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)

    # Add shortcut value to main path, and pass it through a RELU activation
    X = Add()([X_dash,X])
    X = Activation('relu')(X)

    return X


  def convBlock(self, X, filterSize, numFilters, stride=2):
    ''' 
    Arguments:
    ----------
    X => Input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    filterSize => Size of the filter for the middle Conv Layer
    numFilters => Python list of integers, defining the number of filters in the CONV layers
    stride => Integer, specifying the stride to be used

    Output:
    -------
    X => output of the convolutional block, tensor of shape (n_H, n_W, n_C)

    Description:
    ------------

    Following is the identity block:
      X - conv-batchNorm-relu1 - conv-batchNorm-relu2 - conv-batchNorm3 - relu1
      X_dash - conv-batchNorm-relu4 - relu1
    '''

    # Retrieve number of filters for each Conv Layer
    nF1, nF2, nF3 = numFilters

    # Save the input value so that it can be fed directly to the last relu layer of the block
    X_dash = X

    # First component - conv-batchNorm-relu1
    X = Conv2D(filters = nF1, kernel_size = (1, 1), strides = (stride,stride), padding = 'valid', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)

    # Second component - conv-batchNorm-relu2
    X = Conv2D(filters = nF2, kernel_size = (filterSize, filterSize), strides = (1,1), padding = 'same', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)

    # Third component - conv-batchNorm3
    X = Conv2D(filters = nF3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)

    # Shortcut path
    X_dash = Conv2D(filters = nF3, kernel_size = (1, 1), strides = (stride,stride), kernel_initializer=glorot_uniform(seed=0))(X_dash)
    X_dash = BatchNormalization(axis = 3)(X_dash)

    # Add shortcut value to main path, and pass it through a RELU activation
    X = Add()([X, X_dash])
    X = Activation('relu')(X)

    return X



In [ ]:
train_dir = '/content/intel-data/train'
test_dir = '/content/intel-data/test'
resnet = ResNet()
resnet.loadData(train_dir, test_dir, inputShape=(150,150,3), nClasses=6, batchSize=256)
resnet.train(100)

In [ ]:
resnet.model.save_weights('/content/intel-model.h5')